# SRC2 testing

## Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates 
from datetime import datetime
from numpy import linalg as LA
import multiprocessing
from joblib import Parallel, delayed
from datetime import timedelta

import platform
OS = platform.system()

import matplotlib.pyplot as plt
if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows':
    %matplotlib qt
    print('Windows')
elif OS == 'Darwin':
    %matplotlib tk
    print('Mac (Funciona?)')
else:
    print('OS not detected :-|')

import sys
from pathlib import Path
sys.path.insert(1, '../src2/data/')
sys.path.insert(1, '../src2/models/')
sys.path.insert(1, '../src2/utils/')
import toml

from SEIRHVD import SEIRHVD 
import functions
from cv19data import ImportData

Linux


# Importing Data

In [2]:
# First simulation Initial date
initdate = datetime(2020,12,1)
# Date for change in SeroPrevalence
SPchange_date = datetime(2020,7,25)

# Current date
currentdate = datetime.now()
currentday = (currentdate - initdate).days

fiestaspatrias = datetime(2020,9,18)
fiestaspatrias_day = (fiestaspatrias-initdate).days
plebiscito = datetime(2020,10,25)
plebiscito_day = (plebiscito-initdate).days

navidad = datetime(2020,12,25)
navidad_day = (navidad-initdate).days

newyear = datetime(2020,12,31)
newyear_day = (newyear-initdate).days

today = datetime.today()
today_day = (today-initdate).days

In [3]:
print('Importing data from Region Metropolitana')
tstate = '13'#['01','02','03','04','05','06','07','08','09','10','11','12','13','14','15','16']
#tstate = '13101'
# Import Data
state = ImportData(tstate=tstate,initdate = initdate)
state.importdata()

#state.importDailyInfectedNacional()

Importing data from Region Metropolitana
Importing General Data
Importing Population
Importing Active infected
updating database
Importing Accumulated Infected
updating database
Importing Daily Infected
updating database
Importing ICU Beds Data
updating database
Importing Deaths by DEIS
updating database
Done


In [4]:
state.data

,days,dates,I,I_ac,I_d,UCI_capacity,UCI_use_covid,UCI_use_noncovid,D_confirmed,D_suspected,D_ac_confirmed,D_ac_suspected
0,0,2020-12-01,NaN,NaN,531.0,1085.0,321.0,657.0,9.0,7.0,9.0,7.0
1,1,2020-12-02,NaN,NaN,476.0,1087.0,318.0,665.0,18.0,2.0,27.0,9.0
2,2,2020-12-03,NaN,NaN,442.0,1104.0,312.0,670.0,15.0,7.0,42.0,16.0
3,3,2020-12-04,2800.0,332141.0,428.0,1095.0,293.0,689.0,11.0,5.0,53.0,21.0
4,4,2020-12-05,NaN,NaN,438.0,1093.0,281.0,699.0,14.0,5.0,67.0,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...
114,114,2021-03-25,NaN,NaN,2718.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,115,2021-03-26,17659.0,458616.0,2732.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
116,116,2021-03-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,117,2021-03-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
state.plot('days','I')
state.plot('days','I_d')

In [17]:
state.data['I']

0          NaN
1          NaN
2          NaN
3       2800.0
4          NaN
        ...   
110        NaN
111    15515.0
112        NaN
113        NaN
114        NaN
Name: I, Length: 115, dtype: float64

In [25]:
aux = state.data[['days','I']].dropna()
plt.plot(aux['days'],aux['I'])
plt.xlabel('test')
plt.legend(loc=0)
plt.plot

No handles with labels found to put in legend.


<function matplotlib.pyplot.plot(*args, scalex=True, scaley=True, data=None, **kwargs)>

## Config File

# Create SEIRHVD Object

In [4]:
cfg = '/home/samuel/covid19geomodeller/config_files/SEIRHVD.toml'

In [7]:
cfgdata = toml.load(cfg)

In [33]:
import time

In [34]:
start = time.time()
end  = time.time()

1616434394.337443

In [28]:
def a_f(t):
    return 4

In [55]:
times = []
for i in range(100):
    start = time.time()
    c = a + 3 
    end = time.time()
    times.append(end-start)

In [57]:
times_f = []
for i in range(100):
    start = time.time()
    c_f = a_f(10) + 3
    end = time.time()
    times_f.append(end-start)

In [59]:
np.mean(times)

1.6927719116210937e-07

In [60]:
np.mean(times_f)

1.6689300537109374e-07

# SEIRHVD object initialization

In [6]:
import importlib

In [7]:
importlib.reload(plt)

<module 'matplotlib.pyplot' from '/home/samuel/.local/lib/python3.8/site-packages/matplotlib/pyplot.py'>

In [8]:
import SEIRHVD
importlib.reload(SEIRHVD)
from SEIRHVD import SEIRHVD

In [9]:
cfg = '/home/samuel/covid19geomodeller/config_files/SEIRHVD.toml'

In [10]:
sim = SEIRHVD(cfg=cfg)

Importing data
Importing General Data
Importing Population
Dlab Endpoint Error
Importing Population
Importing Active infected
updating database
Importing Accumulated Infected
updating database
Importing Daily Infected
updating database
Importing ICU Beds Data
updating database
Importing Deaths by DEIS
updating database
Done


AttributeError: 'SEIRHVD' object has no attribute 'H_cap'

In [170]:
sim.I

19354.83870967742

In [180]:
sim.integr_sci(0,500,0.1)

UnboundLocalError: local variable 'E_ac0' referenced before assignment

In [172]:
sim.results

,t,dates,S,E,E_d,E_ac,Ias,Imi,Ise,Icr,...,V_sat,V_cap,H_cap,H_need,I_det,I_d_det,I_ac_det,prevalence_total,prevalence_susc,prevalence_detected
0,0,None,973427.419355,5767.741935,2.883871e+03,14419.354839,7741.935484,10645.161290,645.290323,322.451613,...,1.0,2000.0,3000.0,2000.000000,5682.920431,2841.460215,14207.301077,0.048387,0.048101,0.0
1,1,None,971261.049884,6659.154420,1.815539e+03,15586.025764,7475.464727,10615.675336,520.929301,260.308403,...,1.0,2000.0,3000.0,2064.112807,5519.082647,1295.136427,14597.227250,0.049662,0.049368,0.0
2,2,None,968914.955409,7536.057656,1.442766e+03,16813.408774,7305.843484,10688.048403,440.677210,220.206428,...,1.0,2000.0,3000.0,2094.200757,5450.149559,763.155125,15046.557011,0.051131,0.050828,0.0
3,3,None,966511.438800,8301.486397,1.300042e+03,18030.594621,7214.509764,10842.653573,390.868071,195.316799,...,1.0,2000.0,3000.0,2103.590763,5452.634455,600.022473,15547.541233,0.052769,0.052456,0.0
4,4,None,964032.018330,8992.792925,1.249101e+03,19249.461235,7187.086199,11064.493248,362.019870,180.901351,...,1.0,2000.0,3000.0,2101.306230,5509.539605,568.707634,16094.408643,0.054557,0.054234,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,495,None,0.005609,0.001475,1.559355e-12,337441.237729,0.002070,0.006861,0.000056,0.000028,...,1.0,2000.0,3000.0,22369.887415,0.003009,0.000097,313679.223007,1.027582,1.021454,0.0
496,496,None,0.005372,0.001412,1.213352e-12,337441.237729,0.001983,0.006572,0.000054,0.000027,...,1.0,2000.0,3000.0,22369.887370,0.002882,0.000093,313679.223098,1.027582,1.021454,0.0
497,497,None,0.005146,0.001353,1.616781e-12,337441.237729,0.001900,0.006295,0.000052,0.000026,...,1.0,2000.0,3000.0,22369.887326,0.002760,0.000089,313679.223185,1.027582,1.021454,0.0
498,498,None,0.004929,0.001296,2.860468e-12,337441.237729,0.001819,0.006030,0.000050,0.000025,...,1.0,2000.0,3000.0,22369.887284,0.002644,0.000086,313679.223269,1.027582,1.021454,0.0
